<a href="https://colab.research.google.com/github/lakatosgabor/magyar_modszer/blob/main/Magyar_m%C3%B3dszer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.optimize import linear_sum_assignment

# Operátorok és termékek
operators = ["Gipsz Jakab", "Lakatos Gábor", "Teszt Elek"]
products = ["A", "B", "C"]

# Hatékonysági mátrix
efficiency_matrix = np.array([
    [70, 40, 70],  # Gipsz Jakab
    [70, 90, 20],   # Lakatos Gábor, C-t nem gyárthatja
    [43, 20, 80],   # Teszt Elek, B-t nem gyárthatja
])

# Az érvénytelen hozzárendeléseket kizárjuk úgy, hogy a hatékonyságot 0-ra állítjuk
mask_matrix = np.array([
    [1, 1, 1],  # Gipsz Jakab gyárthat mindent
    [1, 1, 0],  # Lakatos Gábor nem gyárthat C-t
    [1, 0, 1],  # Teszt Elek nem gyárthat B-t
])

# A 0 hatékonyságú értékeket a kizárási mátrix alapján beállítjuk
efficiency_matrix = efficiency_matrix * mask_matrix

# Hungarian algorithm alkalmazása (negatív hatékonysággal, mivel a Hungarian algorithm minimális költséget keres)
row_ind, col_ind = linear_sum_assignment(-efficiency_matrix)

# Az eredmény kiírása
assignments = [(operators[i], products[j]) for i, j in zip(row_ind, col_ind)]
total_efficiency = efficiency_matrix[row_ind, col_ind].sum()

print("Optimális hozzárendelés:")
for operator, product in assignments:
    print(f"{operator} gyártsa a(z) {product} terméket.")
print(f"Összhatékonyság: {total_efficiency}")


Optimális hozzárendelés:
Gipsz Jakab gyártsa a(z) A terméket.
Lakatos Gábor gyártsa a(z) B terméket.
Teszt Elek gyártsa a(z) C terméket.
Összhatékonyság: 240
